In [27]:
import evodiff
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm
from pathlib import Path

data = Path('data')
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

device: cuda


In [20]:
train_df = pd.read_csv(data / 'train_ec_311.csv', index_col=0)
test_df = pd.read_csv(data / 'test_ec_311.csv', index_col=0)
df = pd.concat([train_df, test_df])

In [21]:
lengths = df['Sequence'].apply(len).sort_values()
low_bound = lengths[int(0.1 * len(lengths))]
high_bound = lengths[int(0.9 * len(lengths))]

In [23]:
from evodiff.pretrained import OA_DM_38M

checkpoint = OA_DM_38M()
model, collater, tokenizer, scheme = checkpoint
_ = model.to(device)

In [ ]:
from evodiff.generate import generate_oaardm

generated_sequences = []
target_lengths = np.random.randint(low_bound, high_bound, 500)

for length in tqdm(target_lengths):
    tokenized_sample, generated_sequence = generate_oaardm(model, tokenizer, length, batch_size=1, device='cuda')
    generated_sequences.extend(generated_sequence)


print("Generated sequence:", generated_sequences)

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 288/288 [00:02<00:00, 132.76it/s]

 89%|████████▉ | 280/315 [00:02<00:00, 136.36it/s]

In [29]:
with open(data / 'baseline_generated.fasta', "w") as outfile:
        for i, seq in enumerate(generated_sequences):
            header = f">sequence_{i+1}"  # Create a simple header
            outfile.write(header + "\n")
            outfile.write(seq + "\n")